In [1]:
#!/usr/local/bin/python3	

print ('')
print ('** Lab Big Data in Geogrpahic Information Systems **')
print ('*** Module 2: Observational data in climate sciences ***')
print ('')

## general purpose libraries
import numpy as np
#from io import StringIO
import os
import pandas as pd
#import re
#import codecs

## date libraries
import datetime as dtm
from datetime import datetime

## plotting libraries
import matplotlib.pyplot as plt
#from mpl_toolkits.basemap import Basemap



#-------------------------------------------------
#-- get info on database and fiels organization --
#-------------------------------------------------

flist = [os.path.join(path, name) for path, subdirs, files in os.walk("./CRUTEM.4.6.0.0.station_files/") for name in files]
for i in range(0,5):
  print (flist[i])
  
flist=flist[2:]
print (flist[0:5])

nst=len(flist)
print ("Number of stations = ",nst)
print ('')


#- read example obs table 

filein='./CRUTEM.4.6.0.0.station_files/61/616120'
#filein=flist[0]
print (filein)

#- scan text file to get information from the header 

with open(filein, errors='ignore') as f: data = f.readlines()
#data.encode('utf-8').strip()
print(data[3])
print (len(data))

#- find starting line of data matrix

skipi = data.index("Obs:\n")+1
print('skipi=',skipi)

#data = codecs.open(filein,'r', encoding='utf-8', errors='ignore')
#with open(filein, "r") as fp:
#  for line in fp:
#    line = bytes(line, 'utf-8').decode('utf-8', 'ignore')
#skipi = fp.index("Obs:\n")+1    


#- Extract geographical coordinates

lat_line = [line for line in data if "Lat" in line]
lat = str(lat_line).split("=",1)[1] 
lat = float(lat[:-4].strip())

lon_line = [line for line in data if "Long" in line]
lon = str(lon_line).split("=",1)[1] 
lon = float(lon[:-4].strip())

country_line = [line for line in data if "Country" in line]
country = str(country_line).split("=",1)[1] 

#print('**',lat,lon)


#- Extract data matrix

#- note that fields are separated either by a single or a double white space, 
#   which may cause troubles when trying to easily read in the data ...
#   ... using "delimiter=None" takes care of it

yr = np.genfromtxt(filein, skip_header=skipi, delimiter=None, usecols=0, dtype='i4')
print(yr)
nyr = len(yr)
print (nyr,' years of data from ',yr[0],' to ',yr[nyr-1])

mo = np.genfromtxt(filein, skip_header=skipi, filling_values=-99.0, delimiter=None, dtype='str')[:,1:13]  #[rows,cols]

print (yr)
print (mo[0:2,0:3])

#- build axis for station time series
ti0 = datetime.strptime(str(yr[0]), "%Y")
ti1 = datetime.strptime(str(yr[nyr-1]+1), "%Y")
print(ti0,ti1)
stime = pd.date_range(ti0, ti1, freq='M')
print (stime)
print (len(stime))

#- build station time series
xtemps = np.ma.empty([len(stime)],dtype='float')
cy = 0
for yi in range(0,nyr):
  for mi in range (0,12):
    xtemps[cy] = mo[yi,mi]
    cy=cy+1
    
#print (xtemps)   
xtemps[xtemps==-99.0] = np.nan
#print (xtemps)      

#plt.plot(stime,xtemps)
#plt.show()
        
#- Now build a dataframe with unique column name       
        
xdf = pd.DataFrame({'time': stime, filein[-6:]: xtemps})
xdf = xdf.set_index('time')
     
#print (xdf)        
        
#dates = datetime.strptime(dlin, "%Y-%m-%d:%H") # %HH:%MM:%SS")



#ydf = pd.read_csv(filein, skiprows=21, delim_whitespace=True,usecols=[0],header=None)
#print (ydf.dtypes)
#print(ydf.iloc[-1],ydf.iloc[-1])

#tdf = pd.read_csv(filein, skiprows=21, usecols=[1,2,3,4,5,6,7,8,9,10,11,12],header=None,dtype='float',delim_whitespace=True)
#print (tdf)
# delim_whitespace=True accounts for consecutive blanks as a unique separator


#------------------------------------------------------------
#-- Create data container with time axis from 1850 to 2018 --
#------------------------------------------------------------

#- Monthly resolution
#- Number of stations

#- Build common time axis

taxis = pd.date_range('1850-01', '2019-01', freq='M')
nmonths=len(taxis)
#print (taxis)
#print (len(taxis))

#- Build base dataframe for data

df = pd.DataFrame({'time': taxis})
df = df.set_index(['time'])


#- Build base arrays for metadata

ids = np.empty([1],dtype="str") 
olat = np.empty([1],dtype="float") 
olon = np.empty([1],dtype="float") 
ocountry = np.empty([1],dtype="str") 

#- Now loop over the stations build the database stepwise
#- For the final execution make sure to loop over the entire station list

nst_tmp = sum([len(files) for r, d, files in os.walk("/Users/davidgovi/Dropbox/Data_Science/UniMiB/Geo_Lab/CRUTEM.4.6.0.0.station_files")])

for si in range (0,200):  #range(0,nst-1):

  #- Show processing progress

  if (si==1 or si==5 or si==100 or si==1000 or si ==3000 or si ==6000 or si ==9000 or si ==nst_tmp):
    print (si)

  #- open file 
  
  filein=flist[si]
  #print (si,filein[-6:])
  
  #- scan text file to find starting line of data matrix (if data are actually present)
  with open(filein) as f: data = f.readlines()
  skipi = data.index("Obs:\n")+1
  if (len(data)-skipi < 2):
    continue
  
  #- read in data & apply some filters at the same time
  
  yr = np.genfromtxt(filein, skip_header=skipi, delimiter=None, usecols=0, dtype='i4')
  nyr = len(yr)
  if (nyr < 15):
    continue  # this filters out data series that are too short 
  if (yr[0]>1990 or yr[nyr-1]<1960):
    continue  # do not retain series not covering at least in part the reference period      
  mo = np.genfromtxt(filein, skip_header=skipi, filling_values=-99.0, delimiter=None, dtype='str')[:,1:13]  #[rows,cols]
  #print (nyr,' years of data from ',yr[0],' to ',yr[nyr-1])

  #- build local time axis

  ti0 = datetime.strptime(str(yr[0]), "%Y")
  ti1 = datetime.strptime(str(yr[nyr-1]+1), "%Y")
  stime = pd.date_range(ti0, ti1, freq='M')               

  #- build local temperature time series & clean
    
  xtemps = np.ma.empty([len(stime)],dtype='float')
  cy = 0
  for yi in range(0,nyr):
    for mi in range (0,12):
      xtemps[cy] = mo[yi,mi]
      cy=cy+1    
  xtemps[xtemps==-99.0] = np.nan
 
  #- build local station dataframe
  
  xdf = pd.DataFrame({'time': stime, filein[-6:]: xtemps})
  xdf = xdf.set_index('time')
  
  #- merge into global dataframe

  df = pd.merge(df, xdf, on='time', how='left')

  #- store metadata
  #- station name & coordinates, country
  
  ids = np.append(ids,filein[-6:])
  
  lat_line = [line for line in data if "Lat=" in line]
  lat = str(lat_line).split("=",1)[1] 
  lat = float(lat[:-4].strip())
  olat = np.append(olat,lat)

  lon_line = [line for line in data if "Long=" in line]
  lon = str(lon_line).split("=",1)[1] 
  lon = -float(lon[:-4].strip())      # all longitudes have the wrong sign... correct with -
  olon = np.append(olon,lon)

  country_line = [line for line in data if "Country" in line]
  country = str(country_line).split("=",1)[1] 
  country  = country [:-4].strip()
  ocountry = np.append(ocountry,country)

#print (df)
print(ids)
print(olat)

#- arrange metadata

#- remove leading empty element
ids = ids[1:]
olat = olat[1:]
olon = olon[1:]
ocountry = ocountry[1:]

#- put into dataframe

mdf = pd.DataFrame({'ids': ids, 'lon': olon, 'lat': olat, 'country': ocountry})
mdf = mdf.set_index('ids')

print(mdf)

#stop


#-----------------------------------------------
#-- Data treatment and visualization / Part 1 --
#-----------------------------------------------

#-- (1) Plot station locations --

import cartopy.crs as ccrs
import cartopy.feature as cfeature

ax = plt.axes(projection=ccrs.PlateCarree())
#ax = plt.axes(projection=ccrs.Robinson())

plt.title('Surface temperature measurements - global\n\n')

#- Define underlying map features

ax.set_global()
#ax.set_extent([-180,180,-90,90],crs=ccrs.Geodetic()) #(x0, x1, y0, y1) #lon here is [-180,+180] instead of [0,360]

#ax.stock_img()
ax.coastlines()

ax.add_feature(cfeature.LAND)
#ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, edgecolor='gray')

ax.gridlines(draw_labels=True)

#- Plot data

plt.plot([olon], [olat],
         color='red', linewidth=2, marker='o', markersize=1,
         transform=ccrs.Geodetic())

# Save the plot by calling plt.savefig() BEFORE plt.show()
plt.savefig('Module2_plot01.pdf')
#plt.show()
plt.close()


#-- (2) Plot some time series of randomly chosen stations --

#- Set multipanel plot
fig = plt.figure(figsize=(9,15))

subplots = (4,2)
n_panels = subplots[0] * subplots[1]

proj = ccrs.PlateCarree()

#- Select sites
#rssite = [1,2,3,4]
rssite = np.random.randint(len(df.columns), size=4)
ssite = ids[rssite]
print(ssite)

for fi, f in enumerate(ssite):

    #- locate on map
    ax = fig.add_subplot(subplots[0], subplots[1], (fi*2)+1, projection=proj)
    ax.set_title(' / '.join([mdf.country[ssite[fi]], str(mdf.lon[ssite[fi]]) ] ), {"fontsize" : 9})
    ax.stock_img()
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS)
    plt.plot(mdf.lon[ssite[fi]], mdf.lat[ssite[fi]],
         color='red', marker='o', markersize=5,
         transform=ccrs.Geodetic())         
 
    #- time series
    tser = fig.add_subplot(subplots[0], subplots[1], (fi+1)*2)
    df[ssite[fi]].plot()   

#fig.tight_layout()
plt.savefig('Module2_plot02.pdf')
#plt.show()
plt.close()


** Lab Big Data in Geogrpahic Information Systems **
*** Module 2: Observational data in climate sciences ***

./CRUTEM.4.6.0.0.station_files/.DS_Store
./CRUTEM.4.6.0.0.station_files/61/616120
./CRUTEM.4.6.0.0.station_files/61/612913
./CRUTEM.4.6.0.0.station_files/61/612770
./CRUTEM.4.6.0.0.station_files/61/614010
['./CRUTEM.4.6.0.0.station_files/61/612913', './CRUTEM.4.6.0.0.station_files/61/612770', './CRUTEM.4.6.0.0.station_files/61/614010', './CRUTEM.4.6.0.0.station_files/61/618470', './CRUTEM.4.6.0.0.station_files/61/614420']
Number of stations =  10294

./CRUTEM.4.6.0.0.station_files/61/616120
Lat=   16.6

89
skipi= 21
[1951 1952 1953 1954 1955 1956 1957 1958 1959 1960 1961 1962 1963 1964
 1965 1966 1967 1968 1969 1970 1971 1972 1973 1974 1975 1976 1977 1978
 1979 1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990 1991 1992
 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005 2006
 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018]
68  years of data fr

In [2]:
#-----------------------------------------------
#-- Data treatment and visualization / Part 2 --
#-----------------------------------------------

#-- MAKE SURE TO PROPERLY COMMENT YOUR CODE !! -- 

#-- Make a compressed file (gz or zip) with the script and the plots produced for Task 2
#-- Call it "YOURSURNAME_module2.*"
#-- Send by email (samuel.albani@unimib.it) with object "Big Data Lab in GIS - Module 2" 
#-- Upon receiving the email with attachments I will register your presence to today's lab session
#-- For those who physically attended the lab: send the email by the end of the class at 15:30
#-- For those who have specific situations (student-worker, stage abroad): contact me directly ASAP
#-- This exercise will be considered for the final evaluation, according to the criteria previously explained

#- Apply filters to data
#- Apply transformations: anomaly
#- Work on spatial component: regridding

# Task 2.1 --
# clean a bit the time series

#- only stations with complete reference period [1961-1990]
#- calculate anomaly at monthly resolution wrt reference period


# https://towardsdatascience.com/basic-time-series-manipulation-with-pandas-4432afee64ea

# task 2.1

In [3]:
#resetto l'indice in modo da avere 'time' come colonna
df = df.reset_index()

In [4]:
#definisco, e successivamente applico, una breve funzione che dal timestamp contenuto nella colonna 'time' estragga
#soltanto l'informazione relativa all'anno
def extract_year(timest):
    anno= timest.year
    return anno

In [5]:
df['year'] = df['time'].map(lambda x: extract_year(x))

In [6]:
#filtro, utilizzando l'informazione estratta in precedenza, le serie temporali mantenendo soltanto quelle comprese
#tra il 1961 e il 1990
df = df[df['year']>=1961]
df = df[df['year']<=1990]
df

,time,612913,612770,614010,618470,614420,618810,619900,617660,612912,...,956400,952850,959040,956250,955270,956000,959070,590960,599810,year
1332,1961-01-31,24.6,22.2,15.7,21.0,20.9,25.2,26.5,25.1,NaN,...,25.1,NaN,NaN,27.8,25.2,NaN,NaN,10.3,22.8,1961
1333,1961-02-28,27.6,24.3,19.6,23.0,23.9,26.9,26.8,26.7,NaN,...,23.1,NaN,NaN,25.8,26.3,NaN,NaN,11.4,23.9,1961
1334,1961-03-31,30.9,28.1,21.9,25.1,25.1,28.5,25.8,26.9,29.5,...,21.5,NaN,NaN,23.6,21.7,NaN,NaN,16.0,25.9,1961
1335,1961-04-30,32.8,32.6,24.1,25.0,28.3,27.7,24.9,26.9,31.8,...,14.3,NaN,NaN,16.2,19.3,NaN,NaN,20.5,27.6,1961
1336,1961-05-31,30.9,32.5,27.0,25.0,27.1,27.6,24.0,28.2,31.4,...,12.4,NaN,NaN,14.9,13.7,NaN,NaN,24.2,28.6,1961
1337,1961-06-30,29.1,31.0,25.8,23.7,29.5,26.0,22.8,27.5,29.8,...,10.2,NaN,NaN,12.1,11.5,NaN,NaN,26.8,28.4,1961
1338,1961-07-31,25.6,27.7,34.9,22.5,28.7,24.8,21.3,25.9,26.6,...,8.7,NaN,NaN,10.3,9.7,NaN,NaN,27.7,28.9,1961
1339,1961-08-31,24.8,26.4,34.0,22.2,29.7,24.7,20.7,25.9,25.7,...,9.2,NaN,NaN,11.2,11.3,NaN,NaN,27.0,28.6,1961
1340,1961-09-30,25.4,27.0,30.1,23.9,30.4,25.5,21.3,26.4,26.2,...,11.8,NaN,NaN,14.6,16.0,NaN,NaN,25.4,27.7,1961
1341,1961-10-31,28.9,29.5,24.9,24.1,28.4,26.1,22.5,27.3,27.9,...,13.6,NaN,NaN,17.4,21.5,NaN,NaN,21.9,26.6,1961


In [7]:
#elimino tutte le le colonne relative alle stazioni meteorologiche contenenti valori NaN
df = df.dropna(axis='columns')
df

,time,612770,612650,612260,612850,612930,612330,612300,612910,612570,612230,619010,617010,612960,612500,957530,957620,955270,599810,year
1332,1961-01-31,22.2,21.7,22.6,25.6,21.6,22.1,20.7,24.4,23.4,19.6,18.4,23.6,24.0,22.6,21.7,22.3,25.2,22.8,1961
1333,1961-02-28,24.3,24.0,24.3,28.6,23.1,24.6,22.9,26.4,26.8,21.4,19.7,24.9,26.0,23.7,21.5,23.0,26.3,23.9,1961
1334,1961-03-31,28.1,27.5,28.1,31.2,26.9,27.9,27.7,29.4,30.0,24.8,19.8,25.4,29.3,27.1,20.7,19.8,21.7,25.9,1961
1335,1961-04-30,32.6,31.4,30.7,32.5,31.3,30.2,30.8,31.7,33.3,30.0,19.2,26.4,31.1,31.6,17.6,17.8,19.3,27.6,1961
1336,1961-05-31,32.5,33.6,36.7,33.6,31.0,34.4,34.9,31.4,35.2,33.8,18.0,26.9,30.2,35.2,12.4,12.4,13.7,28.6,1961
1337,1961-06-30,31.0,31.7,35.3,29.8,29.2,32.7,32.9,29.9,32.4,33.1,17.0,27.4,27.9,34.4,10.5,10.2,11.5,28.4,1961
1338,1961-07-31,27.7,28.4,32.6,27.0,25.9,29.3,28.5,26.6,27.9,32.2,16.0,26.7,25.3,30.7,9.6,8.6,9.7,28.9,1961
1339,1961-08-31,26.4,27.4,30.6,26.3,25.4,27.9,27.3,25.8,27.5,29.0,15.6,26.1,25.4,28.8,11.1,10.5,11.3,28.6,1961
1340,1961-09-30,27.0,28.2,32.7,26.8,26.1,29.1,28.7,26.0,27.8,30.1,15.6,26.7,25.8,30.9,14.2,13.5,16.0,27.7,1961
1341,1961-10-31,29.5,28.8,32.8,27.9,27.3,30.7,31.1,27.9,30.1,29.5,15.4,26.5,27.7,30.6,19.0,19.2,21.5,26.6,1961


In [10]:
#droppo le colonne 'time' e 'year' in modo che non interferiscano col calcolo.
#per ogni stazione calcolo la media e la sottraggo al valore del periodo preso in osservazione
df.iloc[:,1:-1].apply(lambda x:x-np.mean(x))

,612770,612650,612260,612850,612930,612330,612300,612910,612570,612230,619010,617010,612960,612500,957530,957620,955270,599810
1332,-6.216111,-7.032222,-7.620278,-2.918056,-5.535833,-6.515,-7.849444,-3.576944,-6.029444,-8.760556,0.278333,-2.353611,-3.186667,-7.337222,4.685,5.316944,6.369167,-4.050278
1333,-4.116111,-4.732222,-5.920278,0.081944,-4.035833,-4.015,-5.649444,-1.576944,-2.629444,-6.960556,1.578333,-1.053611,-1.186667,-6.237222,4.485,6.016944,7.469167,-2.950278
1334,-0.316111,-1.232222,-2.120278,2.681944,-0.235833,-0.715,-0.849444,1.423056,0.570556,-3.560556,1.678333,-0.553611,2.113333,-2.837222,3.685,2.816944,2.869167,-0.950278
1335,4.183889,2.667778,0.479722,3.981944,4.164167,1.585,2.250556,3.723056,3.870556,1.639444,1.078333,0.446389,3.913333,1.662778,0.585,0.816944,0.469167,0.749722
1336,4.083889,4.867778,6.479722,5.081944,3.864167,5.785,6.350556,3.423056,5.770556,5.439444,-0.121667,0.946389,3.013333,5.262778,-4.615,-4.583056,-5.130833,1.749722
1337,2.583889,2.967778,5.079722,1.281944,2.064167,4.085,4.350556,1.923056,2.970556,4.739444,-1.121667,1.446389,0.713333,4.462778,-6.515,-6.783056,-7.330833,1.549722
1338,-0.716111,-0.332222,2.379722,-1.518056,-1.235833,0.685,-0.049444,-1.376944,-1.529444,3.839444,-2.121667,0.746389,-1.886667,0.762778,-7.415,-8.383056,-9.130833,2.049722
1339,-2.016111,-1.332222,0.379722,-2.218056,-1.735833,-0.715,-1.249444,-2.176944,-1.929444,0.639444,-2.521667,0.146389,-1.786667,-1.137222,-5.915,-6.483056,-7.530833,1.749722
1340,-1.416111,-0.532222,2.479722,-1.718056,-1.035833,0.485,0.150556,-1.976944,-1.629444,1.739444,-2.521667,0.746389,-1.386667,0.962778,-2.815,-3.483056,-2.830833,0.849722
1341,1.083889,0.067778,2.579722,-0.618056,0.164167,2.085,2.550556,-0.076944,0.670556,1.139444,-2.721667,0.546389,0.513333,0.662778,1.985,2.216944,2.669167,-0.250278


# task 2.2

Il task 2.2 non è stato eseguito entro la fine del laboratorio.
Il ragionamento che avrei fatto sarebbe stato di arrotondare i valori di latitudine e longitudine delle varie stazioni in modo da ricondurle a punti comuni aggregati.
Dopo di che avrei raggruppato le serie temporali sull'anno, recuperando l'informazione utilizzata al task 2.1
Infine avrei calcolato la media globale di tutte le stazioni aggregate in precedenza, anno per anno in maniera da plottare l'andamento nel tempo. Avrei affiancato questo plot con una mappa cloropletica per mostrare quali aree concorrono, e in che modo, alla media mostrata.